## Running the VUS cooccurence workflow via WES on Cavatica
This notebook tests runs the BRCA Challenge federated_analysis on the Kids First Osteosarcoma study on Cavatica.


In [1]:
from fasp.workflow import cavaticaWESClient, sb

cl = cavaticaWESClient('forei/fasp-vus', debug=True)

/Users/forei/imfenvs/fasp/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Setting up the WES run
This uses the DRS ids of each file to specify the inputs to the cooccurrence analysis.

In [2]:
project = 'forei/fasp-vus'
instances = { "KF":  {
    "wesClient": cavaticaWESClient(project)
    "worklow_url":"sbg://forei/fasp-vus/cooccurrence/2",
    'vcf_file': {
        'path': 'drs://cavatica-ga4gh-api.sbgenomics.com/61848448d2f88b031ebde426',
        'name': 'ab010347-8ae3-4c36-833b-9743e6157524.CGP.filtered.deNovo.vep.merged.vcf.gz'},
    'pathogenicity_file': {
        'path': 'drs://cavatica-ga4gh-api.sbgenomics.com/618475b3d2f88b031ebd8de4',
        'name': 'clinvar_BRCA2_mixed.tsv'}
    },

"bdc":  {
    "wesClient": sbWESClient('https://ga4gh-api.sb.biodatacatalyst.nhlbi.nih.gov/ga4gh/wes/v1', project,
                     '~/.keys/sbbdc_key.json')
    "worklow_url":"sbg://forei/fasp-vus/cooccurrence/2",
    'vcf_file': {
        'path': 'drs://ga4gh-api.sb.biodatacatalyst.nhlbi.nih.gov/617c77bce6261a31b6d12f0a',
        'name': 'COPDGene_phs000951_TOPMed_WGS_freeze.8.chr13.hg38.c1.filtered.vcf'},
    'pathogenicity_file': {
        'path': 'drs://ga4gh-api.sb.biodatacatalyst.nhlbi.nih.gov/617c5ce6e6261a31b6d12ddc',
        'name': 'clinvar_brca2.tsv'}
    }

    
}

In [1]:
params = {
    "project": "forei/fasp-vus",
    "inputs": {
        'vcf_file': {'path': 'drs://ga4gh-api.sb.biodatacatalyst.nhlbi.nih.gov/617c77bce6261a31b6d12f0a',
                     'name': 'COPDGene_phs000951_TOPMed_WGS_freeze.8.chr13.hg38.c1.filtered.vcf',
                     'class': 'File'},

        'pathogenicity_file': {'path': 'drs://ga4gh-api.sb.biodatacatalyst.nhlbi.nih.gov/617c5ce6e6261a31b6d12ddc',
                               'name': 'clinvar_brca2.tsv',
                               'class': 'File'}
        }
    }




SyntaxError: unmatched '}' (3533571355.py, line 8)

In [2]:
params = {
    "project": "forei/fasp-vus",
    "inputs": {
    'vcf_file': {'path': 'drs://cavatica-ga4gh-api.sbgenomics.com/61848448d2f88b031ebde426',
     'name': 'ab010347-8ae3-4c36-833b-9743e6157524.CGP.filtered.deNovo.vep.merged.vcf.gz',
     'class': 'File'},
    'pathogenicity_file': {'path': 'drs://cavatica-ga4gh-api.sbgenomics.com/618475b3d2f88b031ebd8de4',
     'name': 'clinvar_BRCA2_mixed.tsv',
     'class': 'File'}
    }
}


Now we have formulated the body in the way that it can be passed to a client function as follows.

In [3]:
import json
run_id= cl.runGenericWorkflow(
    workflow_url="sbg://forei/fasp-vus/cooccurrence/2",
    workflow_params = json.dumps(params),
    workflow_type = "CWL",
    workflow_type_version = "sbg:draft-2",
    verbose=False
)
run_id

'527f63d1-5a55-40bc-930d-bc7411e706a7'

#### Check until complete (about 4 mins, your time may vary).

In [5]:
cl.getTaskStatus(run_id)

'COMPLETE'

## Getting the results - via DRS
Once the run is complete, further steps can use DRS to obtain the file output from the workflow.

In [7]:
runLog = cl.GetRunLog(run_id)

{'vpi_file': {'path': 'drs://cavatica-ga4gh-api.sbgenomics.com/6188354bd2f88b031ee29d83',
  'name': '_4_BRCA2-vpi.json',
  'class': 'File'},
 'pathology_output': None,
 'all_file': {'path': 'drs://cavatica-ga4gh-api.sbgenomics.com/6188354bd2f88b031ee29d8a',
  'name': '_4_BRCA2-all.json',
  'class': 'File'},
 'out_file': {'path': 'drs://cavatica-ga4gh-api.sbgenomics.com/6188354bd2f88b031ee29d82',
  'name': '_16_BRCA2-cooccurrences.json',
  'class': 'File'},
 'ipv_file': {'path': 'drs://cavatica-ga4gh-api.sbgenomics.com/6188354bd2f88b031ee29d84',
  'name': '_4_BRCA2-ipv.json',
  'class': 'File'},
 'tout_file': {'path': 'drs://cavatica-ga4gh-api.sbgenomics.com/6188354bd2f88b031ee29d88',
  'name': '_4_BRCA2-tout.json',
  'class': 'File'}}

In [8]:
resultsDRSID = runLog['outputs']['out_file']['path']
resultsDRSID

'drs://cavatica-ga4gh-api.sbgenomics.com/6188354bd2f88b031ee29d82'

Use the Cavatica DRS Server to retrieve the results files

In [9]:
from fasp.loc import cavaticaDRSClient
drsClient = cavaticaDRSClient('~/.keys/sevenbridges_keys.json', 's3')

### DRS GetObject

In [10]:
sbDRSID = resultsDRSID.split('/')[-1]

fileDetails = drsClient.getObject(sbDRSID)
fileDetails

{'id': '6188354bd2f88b031ee29d82',
 'name': '_16_BRCA2-cooccurrences.json',
 'size': 21058,
 'checksums': [{'type': 'etag',
   'checksum': '7527cbd2250415cbc1d307c4fb7a1ad9-1'}],
 'self_uri': 'drs://cavatica-ga4gh-api.sbgenomics.com/6188354bd2f88b031ee29d82',
 'created_time': '2021-11-07T20:21:31Z',
 'updated_time': '2021-11-07T20:21:31Z',
 'mime_type': 'application/json',
 'access_methods': [{'type': 's3',
   'region': 'us-east-1',
   'access_id': 'aws-us-east-1'}]}

In [11]:
url = drsClient.getAccessURL(sbDRSID,'s3')

### Downloading the file
Now we can use the url obtained to download the file. We'll create a small function to encapsulate the download.

In [12]:
import requests
import os
def download(url, file_path):
    with open(os.path.expanduser(file_path), "wb") as file:
        response = requests.get(url)
        file.write(response.content)

In [17]:
fullPath =  fileDetails['name']
download(url, fullPath)


In [18]:
import json
with open(fullPath) as json_file:
    data = json.load(json_file)
# delete the local copy of the file
os.remove(fullPath)
# and look at the contents
data

{'cooccurring vus': {"('13', 32315831, 'G', 'A')": {'likelihood data': {'p1': 0.015151515151515152,
    'p2': 0.001,
    'n': 28,
    'k': 1,
    'likelihood': 0.09701474531924571},
   'allele frequencies': {'maxPop': nan,
    'maxPopFreq': nan,
    'cohortFreq': 0.8484848484848485},
   'pathogenic variants': [['13', 32376792, 'G', 'T']]},
  "('13', 32318080, 'C', 'T')": {'likelihood data': {'p1': 0.015151515151515152,
    'p2': 0.001,
    'n': 33,
    'k': 1,
    'likelihood': 0.10418809168519826},
   'allele frequencies': {'maxPop': nan, 'maxPopFreq': nan, 'cohortFreq': 1.0},
   'pathogenic variants': [['13', 32376792, 'G', 'T']]},
  "('13', 32318598, 'T', 'C')": {'likelihood data': {'p1': 0.015151515151515152,
    'p2': 0.001,
    'n': 32,
    'k': 1,
    'likelihood': 0.10271219643185438},
   'allele frequencies': {'maxPop': nan,
    'maxPopFreq': nan,
    'cohortFreq': 0.9696969696969697},
   'pathogenic variants': [['13', 32376792, 'G', 'T']]},
  "('13', 32321240, 'G', 'C')": {'l

In [19]:
# flatten cooccurrence output
flat_vus = []
for k, v in data['cooccurring vus'].items():
    pathogenic_count = len(v['pathogenic variants'])
    ## this is a pythonic way of merging dicts - it is cryptic
    z = {**{"vus":k}, **v['likelihood data'], **v['allele frequencies'], **{"no_pathogenic_coocurrs":pathogenic_count}}
    flat_vus.append(z)

# turn the array of dicts into a data frame    
import pandas as pd
flat_df = pd.DataFrame(flat_vus)
flat_df

,vus,p1,p2,n,k,likelihood,maxPop,maxPopFreq,cohortFreq,no_pathogenic_coocurrs
0,"('13', 32315831, 'G', 'A')",0.015152,0.001,28,1,0.097015,NaN,NaN,0.848485,1
1,"('13', 32318080, 'C', 'T')",0.015152,0.001,33,1,0.104188,NaN,NaN,1.000000,1
2,"('13', 32318598, 'T', 'C')",0.015152,0.001,32,1,0.102712,NaN,NaN,0.969697,1
3,"('13', 32321240, 'G', 'C')",0.015152,0.001,33,1,0.104188,NaN,NaN,1.000000,1
4,"('13', 32325741, 'C', 'T')",0.015152,0.001,33,1,0.104188,NaN,NaN,1.000000,1
5,"('13', 32331128, 'G', 'A')",0.015152,0.001,33,1,0.104188,NaN,NaN,1.000000,1
6,"('13', 32333969, 'A', 'G')",0.015152,0.001,33,1,0.104188,NaN,NaN,1.000000,1
7,"('13', 32338918, 'A', 'G')",0.015152,0.001,33,1,0.104188,NaN,NaN,1.000000,1
8,"('13', 32340868, 'G', 'C')",0.015152,0.001,33,1,0.104188,NaN,NaN,1.000000,1
9,"('13', 32342270, 'CAAA', 'CA')",0.015152,0.001,24,1,0.091633,NaN,NaN,0.727273,1


## To do
- Submit the pathogenicity file from the local system
- Either  access the gnomad file from Gnomad, or supply it from the local system

### Done
- Make the container available to other WES servers by adding the Docker container to Docker Hub instead of the Seven Bridges docker repository
